In [5]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os, sys
import itertools
sys.path.append("/mnt/ssd/NMRlipids_VI-NewIonModel/scripts/")
import calcOrderParameters as cop

In [22]:
exper_fname = "/mnt/ssd/NMRlipids_VI-NewIonModel/Data/POPC/exp/POPC_Order_Parameters_Experiments_all.dat"

sim_l14_fname = "NpT/sim0_ref_noIons/OrdPars.dat"
sim_ecc_fname1 = "NpT/sim22a_0mM_NaCl_OPC3_non-tail-atoms_q-sc_0.80_sig-sc_0.89_noIons-ref/OrdPars.dat"
sim_ecc_fname2 = "NpT/sim22a_0mM_NaCl_OPC4_non-tail-atoms_q-sc_0.80_sig-sc_0.89_noIons-ref/OrdPars.dat"
sim_ecc_fname3 = "NpT/sim22a_0mM_NaCl_SPCE_non-tail-atoms_q-sc_0.80_sig-sc_0.89_noIons-ref/OrdPars.dat"

fnames = [exper_fname, sim_l14_fname, sim_ecc_fname1] #, sim_ecc_fname2, sim_ecc_fname3] 

models = []
for fname in fnames:
    model = cop.parse_op_input(fname)
    models.append(model)


In [23]:
# define mapping for ordering of my OP labels
def popc_order(s):
    groups = ('gamma', 'beta', 'alpha', 'g3', 'g2', 'g1') #, 'palmitoyl', 'oleoyl')
    if not s.startswith(groups):
        return len(groups)
    else:
        for i,g in enumerate(groups):
            if s.startswith(g):
                return i
    
refmodel = models[0]
ax_x_labels = refmodel.keys()
ax_x_labels.sort()
ax_x_labels = sorted(ax_x_labels, key=popc_order)

xlabels = (r'$\gamma$', r'$\beta$', r'$\alpha$', r'$g_3$', r'$g_2$', r'$g_1$')

Plotting figure 1

In [24]:
font = {'family' : 'DejaVu',
        'weight' : 'normal',
        'size'   : 14}

matplotlib.rc('font', **font)

fig, ax = plt.subplots()
# add some text for labels, title and axes ticks
ax.set_xlabel(r'POPC Headgroup and tail order parameters')
ax.set_ylabel(r'$-S_{CD}$')
ax.set_ylim([-0.15,0.3])
r1 = range(len(xlabels))
r2 = range(len(xlabels)+2, len(xlabels)+18)
r1.extend(r2)
ax.set_xticks(r1)
xlabelsl = list(xlabels)
xlabelsl.extend(range(2, 18))
ax.set_xticklabels(xlabelsl, rotation=0)

legends = ['experiment', 'Lipid14, TIP3p', r'ECC-lipids 17, OPC3', r'ECC-lipids 17, OPC4', r'ECC-lipids 17, SPC/E'] #, 'scaled Oxy. sigmas 0.90 0mM', 'scaled OCH headgr. sigmas 0.94 0mM',
           #'scaled 0.75 Oxy. sigmas 0.90 0mM', 'scaled 0.75 Oxy. sigmas 0.90 0mM NpAT']
colours = ['black', '#33AA66',  '#DE7845',  '#DE4578',  '#BB5487']  # zip() truncates the to the shortest list

for (model, legend, colour) in zip(models, legends, colours):
    if "exper" in legend:
        point_marker = "D"
        point_size = 400
    else:
        point_marker = "o"
        point_size = 200
    for (i, label) in enumerate(ax_x_labels):
        if i == 0:
            point_label = legend
        else:
            point_label = ""
        if label in model.keys() and not (label.startswith("ole") or label.startswith("palm")):
            ax.scatter(popc_order(label), -float(model[label].avg), marker=point_marker, c=colour, s=point_size, lw=0, label=point_label)
        else:
            print "Label {lab} not found in current model keys, model: {mod}".format(lab=label, mod=model)
    tails={}
    tails_list={}
    for tail,chiral,i in itertools.product(['palmitoyl_C', 'oleoyl_C'], ['a', 'b'], range(2, 18)):
        label = tail+str(i)+chiral
        point_size = 100
        point_marker = "."
        try: 
            ax.scatter(popc_order(label)+i, -float(model[label].avg), marker=point_marker, c=colour, s=point_size, lw=0, label=point_label)
        except:
            print "Label {lab} not found in current model keys, model: {mod}".format(lab=label, mod=model)
        if not tail in tails.keys():
            tails[tail] = {}
            tails_list[tail] = []
        if label in model.keys():
            if not i in tails[tail].keys():
                tails[tail][i] = []
            try:
                tails[tail][i].append(float(model[label].avg))
            except:
                print "Label {lab} not found in current model keys, model: {mod}".format(lab=label, mod=model)
    for tail in tails.keys():
        for item in tails[tail].items():
            tails_list[tail].append([item[0], np.average(item[1])])
        tail_temp = np.array(tails_list[tail])
        try:
            ax.plot(tail_temp[:,0]+len(xlabels), -tail_temp[:,1], c=colour, lw=1.0, label=point_label)
        except:
            pass
    
            


#ax.legend(loc="upper left", markerscale=0.5, scatterpoints=4)
plt.savefig("Order-parameters_exp-L14-ECCL17_q80_sig89.png", dpi=200)
plt.show()


Label oleoyl_C10a not found in current model keys, model: {'palmitoyl_C11b': <calcOrderParameters.OrderParameter instance at 0x7fb49b503dd0>, 'oleoyl_C13b': <calcOrderParameters.OrderParameter instance at 0x7fb498291c20>, 'oleoyl_C13a': <calcOrderParameters.OrderParameter instance at 0x7fb498291950>, 'oleoyl_C17b': <calcOrderParameters.OrderParameter instance at 0x7fb4982faea8>, 'oleoyl_C17a': <calcOrderParameters.OrderParameter instance at 0x7fb4982fab00>, 'oleoyl_C8a': <calcOrderParameters.OrderParameter instance at 0x7fb498291560>, 'oleoyl_C15a': <calcOrderParameters.OrderParameter instance at 0x7fb498291b48>, 'palmitoyl_C3b': <calcOrderParameters.OrderParameter instance at 0x7fb49b503b00>, 'palmitoyl_C3a': <calcOrderParameters.OrderParameter instance at 0x7fb49b503200>, 'oleoyl_C15b': <calcOrderParameters.OrderParameter instance at 0x7fb4982916c8>, 'oleoyl_C11a': <calcOrderParameters.OrderParameter instance at 0x7fb498291bd8>, 'palmitoyl_C15a': <calcOrderParameters.OrderParameter i